In [225]:
import re
import json
import pandas as pd
import os

In [313]:
import sys

In [314]:
sys.path.insert(0, os.path.abspath('../'))

In [315]:
from module import parse

In [507]:
# clean XMLs

def xml_cleaner(file):
    newfile = re.sub('doctest_block', 'paragraph', file)
    newfile = re.sub('<\/?(emphasis|inline|literal|reference|literal_block).*?>', '', newfile)
    return newfile

def clean_xmls(root):
    nufiles = 0
    for subdir, dirs, files in os.walk(root):
        for file in files:
            if file.endswith(".xml"):
                nufiles += 1
                file_path = os.path.join(subdir, file)
                xml_file = open(file_path, 'r').read()
                open(file_path, 'w').write(xml_cleaner(xml_file))
        print(f"Total files cleaned: {nufiles}")

def xml_to_json(input_dir, output_dir):
    for subdir, dirs, files in os.walk(input_dir):
        out_subdir = "/".join(output_dir.split("/")[:3]+subdir.split("/")[3:])
        for file in files:
            if file.endswith(".xml"):
                in_file_path = os.path.join(subdir, file)
                out_file_path = os.path.join(out_subdir, file)[:-3]+'json'
#                 print(out_file_pathout_file_path[:-3]+'json')
                os.makedirs("/".join(out_file_path.split('/')[:-1]), exist_ok=True)
                parse.runner(in_file_path, out_file_path)

In [508]:
clean_xmls('../data/xml_files/')

Total files cleaned: 6
Total files cleaned: 23
Total files cleaned: 24
Total files cleaned: 25
Total files cleaned: 34
Total files cleaned: 35
Total files cleaned: 42
Total files cleaned: 355
Total files cleaned: 378
Total files cleaned: 379
Total files cleaned: 386
Total files cleaned: 407
Total files cleaned: 471
Total files cleaned: 482


In [509]:
xml_to_json('../data/xml_files', '../data/json_files')

In [510]:
RUBBISH = ['@']

def clean(text):
    return re.sub('^\s+|\s+$|\s+(?=\s)', ' ', text.strip())

def parse_dict_to_list(d, level=None, curr_level=0):
    result = []
    if isinstance(d, dict):
        for k, v in d.items():
            if isinstance(v, str) and k[0] not in RUBBISH:
                result.append([k, clean(v)])
            elif isinstance(v, list):
                for item in v:
                    if isinstance(item, str) and k[0] not in RUBBISH:
                        result.append([k, clean(item)])
                    else:
                        sub_result = parse_dict_to_list(item, level, curr_level + 1)
                        if curr_level < level:
                            result.extend(sub_result)
                        else:
                            result.append([k, ' '.join([text for _, text in sub_result])])
            else:
                sub_result = parse_dict_to_list(v, level, curr_level + 1)
                result.extend(sub_result)
        if curr_level==level:
            result=[[d.get('title', ''), '. '.join([text for _, text in result])]]
    return result


In [511]:
def clean_parsed_list(documents):
    result = [x for x in documents if x[0]]
    for item in result:
        item[1] = re.sub('(\.\s?\.\s?)+', '. ', item[1])
    return result

In [512]:
def propogate_dataset(dataset, documents):
    pd_docs = pd.DataFrame(clean_parsed_list(documents), columns=['title', 'text'])
    return pd.concat([dataset, pd_docs], ignore_index=True)

In [513]:
def parse_all_jsons(root, dataset):
    nufiles = 0
    for subdir, dirs, files in os.walk(root):
        for file in files:
            if file.endswith(".json"):
                nufiles += 1
                file_path = os.path.join(subdir, file)
                json_file = json.loads(open(file_path, 'r').read())
                documents = parse_dict_to_list(json_file, level=3)
                dataset = propogate_dataset(dataset, documents)
        print(f"Total files: {nufiles}")
    return dataset

In [514]:
dataset = pd.DataFrame([], columns=['title', 'text'])
dataset = parse_all_jsons('../data/json_files/', dataset)

Total files: 6
Total files: 23
Total files: 24
Total files: 25
Total files: 34
Total files: 35
Total files: 42
Total files: 355
Total files: 378
Total files: 379
Total files: 386
Total files: 407
Total files: 471
Total files: 482


In [515]:
dataset = dataset.drop(dataset[dataset.title.isin(['paragraph', 'title', 'Important'])].index)

In [521]:
dataset

,title,text
4,Documentation bugs,Documentation bugs. If you find a bug in this ...
5,Using the Python issue tracker,Using the Python issue tracker. Issue reports ...
6,Getting started contributing to Python yourself,Getting started contributing to Python yoursel...
9,History of the software,History of the software. Python was created in...
10,Terms and conditions for accessing or otherwis...,Terms and conditions for accessing or otherwis...
...,...,...
3705,Identifiers and keywords,Identifiers and keywords. . Identifiers (also ...
3706,Literals,Literals. Literals are notations for constant ...
3707,Operators,Operators. . The following tokens are operator...
3708,Delimiters,Delimiters. The following tokens serve as deli...


In [532]:
dataset.columns.astype(str)

Index(['title', 'text'], dtype='object')

In [534]:
dataset.astype(str).to_parquet('dataset_cpython.gzip', compression='gzip')